In [2]:
import os
import json
import joblib
import mlflow
import pandas as pd
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

In [3]:
applicants_path = "../../data/raw/applicants.json"
vagas_path = "../../data/raw/vagas.json"
prospects_path = "../../data/raw/prospects.json"

print(f"\n=== Loading JSON files ===")
print(f"Loading applicants from: {applicants_path}")
with open(applicants_path, encoding='utf-8') as f:
    applicants = json.load(f)
print(f"Loading vagas from: {vagas_path}")
with open(vagas_path, encoding='utf-8') as f:
    vagas = json.load(f)
print(f"Loading prospects from: {prospects_path}")
with open(prospects_path, encoding='utf-8') as f:
    prospects = json.load(f)


=== Loading JSON files ===
Loading applicants from: ../../data/raw/applicants.json
Loading vagas from: ../../data/raw/vagas.json
Loading prospects from: ../../data/raw/prospects.json


In [4]:
rows = []
statuses = []
for vaga_idx in prospects:
    if vaga_idx in vagas:
        for prospect_candidate in prospects[vaga_idx]["prospects"]:
            if prospect_candidate["codigo"] in applicants:
                rows.append({
                    "job_description": vagas[vaga_idx]["perfil_vaga"]["principais_atividades"],
                    "job_requirements": vagas[vaga_idx]["perfil_vaga"]["competencia_tecnicas_e_comportamentais"],
                    "candidate_cv": applicants[prospect_candidate["codigo"]]["cv_pt"],
                    "status": prospect_candidate["situacao_candidado"]
                })
                if (prospect_candidate["situacao_candidado"] not in statuses):
                    statuses.append(prospect_candidate["situacao_candidado"])

print(statuses)

data = pd.DataFrame(rows)
print(data)

['Encaminhado ao Requisitante', 'Contratado pela Decision', 'Desistiu', 'Documentação PJ', 'Não Aprovado pelo Cliente', 'Prospect', 'Não Aprovado pelo RH', 'Aprovado', 'Não Aprovado pelo Requisitante', 'Inscrito', 'Entrevista Técnica', 'Em avaliação pelo RH', 'Contratado como Hunting', 'Desistiu da Contratação', 'Entrevista com Cliente', 'Documentação CLT', 'Recusado', 'Documentação Cooperado', 'Sem interesse nesta vaga', 'Encaminhar Proposta', 'Proposta Aceita']
                                         job_description  \
0      - Experiência comprovada em projetos de control-M   
1      - Experiência comprovada em projetos de control-M   
2      Key skills required for the job are:\n\nPeople...   
3      Key skills required for the job are:\n\nPeople...   
4      Arquiteto\n\nFoco na área e automação.\n\nRequ...   
...                                                  ...   
45066  Experiência na implementação de projetos de SD...   
45067  Experiência na implementação de projetos de S

In [5]:
status_map = {
    "Encaminhado ao Requisitante":        0,
    "Contratado pela Decision":           1,
    "Desistiu":                           0,
    "Documentação PJ":                    1,
    "Não Aprovado pelo Cliente":          0,
    "Prospect":                           0,
    "Não Aprovado pelo RH":               0,
    "Aprovado":                           1,
    "Não Aprovado pelo Requisitante":     0,
    "Inscrito":                           0,
    "Entrevista Técnica":                 0,
    "Em avaliação pelo RH":               0,
    "Contratado como Hunting":            1,
    "Desistiu da Contratação":            0,
    "Entrevista com Cliente":             0,
    "Documentação CLT":                   1,
    "Recusado":                           0,
    "Documentação Cooperado":             1,
    "Sem interesse nesta vaga":           0,
    "Encaminhar Proposta":                1,
    "Proposta Aceita":                    1
}


x = data[["job_description","job_requirements","candidate_cv"]]
y = data["status"].map(status_map)


X_train, X_test, y_train, y_test = train_test_split(
            x, y, stratify=y, test_size=0.2, random_state=42
)



In [6]:
preprocessor = ColumnTransformer([
    ("tfidf_ativ", TfidfVectorizer(max_features=1000, ngram_range=(1,2)),
        "job_description"),
    ("tfidf_comp", TfidfVectorizer(max_features=1000, ngram_range=(1,2)),
        "job_requirements"),
    ("tfidf_cv",   TfidfVectorizer(max_features=5000, ngram_range=(1,2)),
        "candidate_cv"),
], remainder="drop")


pipeline = Pipeline([
    ("pre", preprocessor),
    ("clf", LogisticRegression(max_iter=1000))
])

grid = GridSearchCV(
    pipeline, {"clf__C": [0.1, 1, 10]},
    cv=5, scoring="roc_auc", n_jobs=-1
)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('tfidf_ativ',
                                                                         TfidfVectorizer(max_features=1000,
                                                                                         ngram_range=(1,
                                                                                                      2)),
                                                                         'job_description'),
                                                                        ('tfidf_comp',
                                                                         TfidfVectorizer(max_features=1000,
                                                                                         ngram_range=(1,
                                                                                                      2)),
                                                                         'job_requirements'),
                                                                        ('tfidf_cv',
                                                                         TfidfVectorizer(max_features=5000,
                                                                                         ngram_range=(1,
                                                                                                      2)),
                                                                         'candidate_cv')])),
                                       ('clf',
                                        LogisticRegression(max_iter=1000))]),
             n_jobs=-1, param_grid={'clf__C': [0.1, 1, 10]}, scoring='roc_auc')

In [7]:
y_pred = grid.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
print(f"AUC no conjunto de teste: {auc:.4f}")

AUC no conjunto de teste: 0.7910


In [ ]:
model_output    = 'models/agent_match_model.joblib'

os.makedirs(os.path.dirname(model_output), exist_ok=True)
joblib.dump(grid.best_estimator_, model_output)
print(f"Modelo salvo em: {model_output}")